In [1]:
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence
import pandas as pd
import numpy as np
import os

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-05-05 17:30:50.096077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-05 17:30:51.048018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator import OptimizedDataGenerator
from models.models import *

In [3]:
model=CreateModel((13,21,2),n_filters=5,pool_size=3)
model.summary()

2025-05-05 17:30:53.224691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3234 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:41:00.0, compute capability: 8.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 2)]       0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         33        
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [4]:
# get best weights file
pitch = '50x12P5'
batch_size = 5000
fingerprint = '34c2da80'
files = os.listdir('/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_cotBeta1P5_quantize_manual_400_1600_2400_weights/weights-{}-bs{}-{}-checkpoints'.format(pitch, batch_size, fingerprint))

vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
#vlosses = [float(f.split("-v")[1].split("-tr")[0]) for f in files]
bestfile = files[np.argmin(vlosses)]
model.load_weights('/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_cotBeta1P5_quantize_manual_400_1600_2400_weights/weights-{}-bs{}-{}-checkpoints/'.format(pitch, batch_size, fingerprint)+bestfile)

print('Best model: {}'.format(bestfile))

Best model: weights.491-t-17984.29-v-18971.00.hdf5


In [6]:
pitch = '50x12P5'
model.load_weights('/home/dajiang/smart-pixels-ml/weights/best_model_dataset_2s_50x12P5x100_timeslices2_bs5000_weights.hdf5')

In [ ]:
# load in the test set
#test_generator = OptimizedDataGenerator(
#    load_from_tfrecords_dir = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_{}_2t_bs5000_test'.format(pitch),
#    quantize = True
#)


test_generator = OptimizedDataGenerator(
    data_directory_path = '/data/dajiang/smart-pixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/',
    labels_directory_path = '/data/dajiang/smart-pixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/',
    is_directory_recursive = False,
    file_type = 'parquet',
    data_format = '3D',
    batch_size = 5000,
    file_count = 50,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (2,13,21),
    transpose = (0,2,3,1),
    files_from_end = False,
    shuffle = True,

    #load_from_tfrecords_dir = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_bs5000_test',
    tfrecords_dir = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_bs5000_test',
    use_time_stamps = [0,19], #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 1,
    quantize = True # Quantization ON
)

Directory /data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_bs5000_test does not exist and cannot be removed.


Processing Files...:   0%|          | 0/41 [00:00<?, ?it/s]

In [9]:
# predicts test data
p_test = model.predict(test_generator)

complete_truth = None
for _, y in test_generator:
    if complete_truth is None:
        complete_truth = y
    else:
        complete_truth = np.concatenate((complete_truth, y), axis=0)

# creates df with all predicted values and matrix elements - 4 predictions, all 10 unique matrix elements
df = pd.DataFrame(p_test,columns=['x','M11','y','M22','cotA','M33','cotB','M44','M21','M31','M32','M41','M42','M43'])

# stores all true values in same matrix as xtrue, ytrue, etc.
df['xtrue'] = complete_truth[:,0]
df['ytrue'] = complete_truth[:,1]
df['cotAtrue'] = complete_truth[:,2]
df['cotBtrue'] = complete_truth[:,3]
df['M11'] = minval+tf.math.maximum(df['M11'], 0)
df['M22'] = minval+tf.math.maximum(df['M22'], 0)
df['M33'] = minval+tf.math.maximum(df['M33'], 0)
df['M44'] = minval+tf.math.maximum(df['M44'], 0)

df['sigmax'] = abs(df['M11'])
df['sigmay'] = np.sqrt(df['M21']**2 + df['M22']**2)
df['sigmacotA'] = np.sqrt(df['M31']**2+df['M32']**2+df['M33']**2)
df['sigmacotB'] = np.sqrt(df['M41']**2+df['M42']**2+df['M43']**2+df['M44']**2)

# calculates residuals for x, y, cotA, cotB
df['residualsX'] = df['xtrue'] - df['x']
df['residualsY'] = df['ytrue'] - df['y']
df['residualsA'] = df['cotAtrue'] - df['cotA']
df['residualsB'] = df['cotBtrue'] - df['cotB']

# calculates pulls for x, y, cotA, cotB
df['pullx'] = (df['xtrue']-df['x'])/df['sigmax']
df['pully'] = (df['ytrue']-df['y'])/df['sigmay']
df['pullcotA'] = (df['cotAtrue']-df['cotA'])/df['sigmacotA']
df['pullcotB'] = (df['cotBtrue']-df['cotB'])/df['sigmacotB']

# stores results as parquet
df.to_parquet("/home/dajiang/smart-pixels-ml/processed_parquets/{}_bs{}_2t_train-dataset_2s_test-dataset_2s_vars.parquet".format(pitch, batch_size))#, fingerprint))

InvalidArgumentError: Graph execution error:

Detected at node 'model/q_separable_conv2d/separable_conv2d/depthwise' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib